In [ ]:
import h3
import json

def h3ToFeature(index, properties={}):
    coordinates = h3.h3_to_geo_boundary(index, geo_json=True)
    return {
        "type": "Feature",
        "id": index,
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates]
        }
    }

def featuresToCollection(features):
    return json.dumps({
        "type": "FeatureCollection",
        "features": features
    })

In [ ]:
import pandas

H3_RESOLUTION = 9

# Load a spreadsheet and make some nice hexagons
def h3_counts(points: pandas.DataFrame) -> dict:
    result = {}
    for _, row in points.iterrows():
        h3_id = h3.geo_to_h3(row['lat'], row['lon'], H3_RESOLUTION)
        result[h3_id] = result.get(h3_id, 0) + 1

    return result

# Read the CSV, count the points at each hexagon
df = pandas.read_csv("output/brisbane-neuron/1657450947.csv")
counts = h3_counts(df)

# Convert into a geojson file
features = [h3ToFeature(key, { "count": value}) for key, value in counts.items()]
geojson = featuresToCollection(features)

with open("counts.geojson", "w") as f:
    f.write(geojson)

In [ ]:
from keplergl import KeplerGl

config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [],
            "layers": [
                {
                    "id": "d8y0c0e",
                    "type": "geojson",
                    "config": {
                        "dataId": "Bike Counts",
                        "label": "Bike Counts",
                        "color": [241, 92, 23],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {"geojson": "_geojson"},
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "strokeOpacity": 0.8,
                            "thickness": 0.5,
                            "strokeColor": [34, 63, 154],
                            "colorRange": {
                                "name": "ColorBrewer Reds-6",
                                "type": "singlehue",
                                "category": "ColorBrewer",
                                "colors": ["#fee5d9", "#fcbba1", "#fc9272", "#fb6a4a", "#de2d26", "#a50f15"],
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"],
                            },
                            "radius": 10,
                            "sizeRange": [0, 10],
                            "radiusRange": [0, 50],
                            "heightRange": [0, 500],
                            "elevationScale": 5,
                            "enableElevationZoomFactor": True,
                            "stroked": True,
                            "filled": True,
                            "enable3d": True,
                            "wireframe": False,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": {"name": "count", "type": "integer"},
                        "colorScale": "quantile",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                        "heightField": {"name": "count", "type": "integer"},
                        "heightScale": "linear",
                        "radiusField": None,
                        "radiusScale": "linear",
                    },
                }
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {"Bike Counts": [{"name": "count", "format": None}]},
                    "compareMode": False,
                    "compareType": "absolute",
                    "enabled": True,
                },
                "brush": {"size": 0.5, "enabled": False},
                "geocoder": {"enabled": False},
                "coordinate": {"enabled": False},
            },
            "layerBlending": "normal",
            "splitMaps": [],
            "animationConfig": {"currentTime": None, "speed": 1},
        },
        "mapState": {
            "bearing": 22.01574784077301,
            "dragRotate": True,
            "latitude": -27.46438803770352,
            "longitude": 153.06498853005814,
            "pitch": 51.264236550510816,
            "zoom": 10.513064752221112,
            "isSplit": False,
        },
    },
}

map = KeplerGl()
map.add_data(geojson, name="Bike Counts")
map.config = config
map
